In [ ]:
import kserve
import boto3
from typing import List, Dict
import re
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import json



class KFServingExplainModel(kfserving.KFModel):
    def __init__(self, name: str):
        super().__init__(name)
        self.name = name
        self.ready = False
        self.model = None
        self.gpu = True

    def load(self):
        self.model = load_model('final_model.h5')
        self.ready = True

    def predict(self, request: Dict) -> Dict:
        image_link = request['instances'][0]['image_link']
        img = "your_image.jpg"

        session = boto3.session.Session()
        s3_client = session.client(
                    service_name='s3',
                    aws_access_key_id='INPUT S3 ACCESS KEY HERE',
                    aws_secret_access_key='INPUT S3 SECRET KEY HERE',
                    endpoint_url='INPUT S3 CREDENTIALS HERE',
                    region_name='ru-1a'
        )
        s3_client.download_file('INPUT S3 BUCKETNAME CREDENTIALS HERE', image_link, img)

        img = load_img(img, target_size=(224, 224))
        img = img_to_array(img)
        img = img.reshape(1, 224, 224, 3)
        img = img.astype('float32')
        img = img - [123.68, 116.779, 103.939]

        result = self.model.predict(img)
        result = result[0][0]

        if(result >= 0.5):
            result = 'dog'
        elif(result < 0.5):
            result = 'cat'

        return {"predictions": result}


if __name__ == "__main__":
    x = re.compile('(kfserving-\d+)').search(os.environ.get('HOSTNAME'))
    name = "kfserving-default"
    if x:
        name = x[0]

    model = KFServingExplainModel(name)
    model.load()
    kfserving.KFServer(workers=1).start([model])